In [10]:
import numpy as np
import math
import csv

def read_data(filename):
    with open(filename, 'r') as csvfile:
        datareader = csv.reader(csvfile, delimiter=',')
        headers = next(datareader)
        metadata = []
        traindata = []
        for name in headers:
            metadata.append(name)
        for row in datareader:
            traindata.append(row)

    return (metadata, traindata)

class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ""

    def __str__(self):
        return self.attribute

def subtables(data, col, delete):
    dict = {}
    items = np.unique(data[:, col])
    count = np.zeros((items.shape[0], 1), dtype=np.int32)

    for x in range(items.shape[0]):
        for y in range(data.shape[0]):
            if data[y, col] == items[x]:
                count[x] += 1

    for x in range(items.shape[0]):
        dict[items[x]] = np.empty((int(count[x][0]), data.shape[1]), dtype=object)
        pos = 0
        for y in range(data.shape[0]):
            if data[y, col] == items[x]:
                dict[items[x]][pos] = data[y]
                pos += 1
        if delete:
            dict[items[x]] = np.delete(dict[items[x]], col, 1)

    return items, dict

def entropy(S):
    items, counts = np.unique(S, return_counts=True)
    probabilities = counts / counts.sum()
    log_probs = np.log2(probabilities, where=probabilities > 0)
    return -np.sum(probabilities * log_probs)

def gain_ratio(data, col):
    items, dict = subtables(data, col, delete=False)

    total_size = data.shape[0]
    entropies = np.zeros((items.shape[0], 1))
    intrinsic = np.zeros((items.shape[0], 1))

    for x in range(items.shape[0]):
        ratio = dict[items[x]].shape[0] / (total_size * 1.0)
        entropies[x] = ratio * entropy(dict[items[x]][:, -1])
        intrinsic[x] = ratio * math.log2(ratio) if ratio > 0 else 0

    total_entropy = entropy(data[:, -1])
    iv = -1 * sum(intrinsic)

    if iv == 0:
        return 0

    return (total_entropy - sum(entropies)) / iv

def create_node(data, metadata):
    if (np.unique(data[:, -1])).shape[0] == 1:
        node = Node("")
        node.answer = np.unique(data[:, -1])[0]
        return node

    if data.shape[1] == 1: #Added check to avoid error
        node = Node("")
        node.answer = np.unique(data[:,0])[0]
        return node

    gains = np.zeros((data.shape[1] - 1, 1))

    for col in range(data.shape[1] - 1):
        gains[col] = gain_ratio(data, col)

    split = np.argmax(gains)

    node = Node(metadata[split])
    metadata_new = metadata.copy()
    metadata_new = np.delete(metadata_new, split, 0)

    items, dict = subtables(data, split, delete=True)

    for x in range(items.shape[0]):
        child = create_node(dict[items[x]], metadata_new)
        node.children.append((items[x], child))

    return node

def empty(size):
    return "    " * size

def print_tree(node, level):
    if node.answer != "":
        print(empty(level), node.answer)
        return
    print(empty(level), node.attribute)
    for value, n in node.children:
        print(empty(level + 1), value)
        print_tree(n, level + 2)

metadata, traindata = read_data("tennisdata.csv")
data = np.array(traindata, dtype=object)
node = create_node(data, metadata)
print_tree(node, 0)

 Outlook
     Overcast
         Yes
     Rain
         Wind
             Strong
                 Temp
                     Cool
                         Humidity
                             Normal
                                 No
                     Mild
                         No
             Weak
                 Yes
     Sunny
         Humidity
             High
                 No
             Normal
                 Yes
